In [ ]:
# !pip install -r requirements.txt

In [ ]:
import dotenv
import os
import openai
import json
import pandas as pd
import sys
import requests


# .env 파일 로드 (있다면)
dotenv.load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
SLACK_URL = os.getenv("SLACK_URL")

In [46]:
# 작업 종료 후 slack으로 메신저 보내기기
def send_msg(msg):
    url = SLACK_URL
    message = (msg + " 작업 완료") 
    title = (f"New Incoming Message :zap:") # 타이틀 입력
    slack_data = {
        "username": "NotificationBot", # 보내는 사람 이름
        "icon_emoji": ":satellite:",
        #"channel" : "#somerandomcahnnel",
        "attachments": [
            {
                "color": "#9733EE",
                "fields": [
                    {
                        "title": title,
                        "value": message,
                        "short": "false",
                    }
                ]
            }
        ]
    }
    byte_length = str(sys.getsizeof(slack_data))
    headers = {'Content-Type': "application/json", 'Content-Length': byte_length}
    response = requests.post(url, data=json.dumps(slack_data), headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)

In [5]:
df = pd.read_csv("data_detail.csv")
texts = df['text'].to_list()
contents = df['detail'].to_list()

In [6]:
client = openai.OpenAI()

# 최적화된 시스템 프롬프트
OPTIMIZED_SYSTEM_PROMPT = """
당신은 소프트웨어 프로젝트의 요구사항 분석 전문가입니다.
주어진 프로젝트 개요를 바탕으로 현실적이고 구체적인 기능 요구사항과 성능 요구사항을 생성하는 것이 목표입니다.

생성 규칙:
1. 기능 요구사항(FUNCTIONAL): 사용자 관점에서 시스템이 제공해야 하는 구체적인 기능
2. 성능 요구사항(PERFORMANCE): 시스템의 응답시간, 처리량, 가용성, 확장성 등 측정 가능한 수치 포함
3. 각 요구사항은 명확하고 구현 가능하며 테스트 가능해야 함
4. 프로젝트 규모와 성격에 적합한 현실적인 수준이어야 함
5. 반드시 지정된 JSON 형식으로만 응답할 것
"""

# 파인튜닝된 모델 ID
MODEL_ID = 'gpt-4o-mini'

def detail_plus(project_overview,existing_requirements, additional_count = 5):
  # 구조화된 요청 프롬프트 (f-string 사용하지 않음)
    enhanced_prompt = f"""

    프로젝트 주제 :
    {project_overview}

    기존 요구사항 목록:
    {existing_requirements}

    위 기존 주제와 요구사항들을 참고하여 추가 요구사항을 생성해주세요
    또한 이렇게 생성된 요구사항은 원래의 '기존 요구사항 목록'에 추가시켜서 정렬 후 출력하세요:

    **생성 조건:**
    - 기존 주제와 요구사항과 일관성을 유지하면서 새로운 {additional_count}개의 요구사항 생성
    - 기존 주제와 요구사항의 도메인과 맥락을 고려
    - 기능 주제와 요구사항과 성능 요구사항을 적절히 혼합
    - 기존 요구사항과 중복되지 않는 새로운 관점의 요구사항

    **중요: 반드시 아래 JSON 형식으로만 응답하세요. 다른 텍스트, 설명, 주석은 절대 포함하지 마세요.**

    JSON 형식:
    [
      {{"requirementType": "FUNCTIONAL1", "content": "구체적인 기능 요구사항"}},
      {{"requirementType": "FUNCTIONAL2", "content": "구체적인 기능 요구사항"}},
      {{"requirementType": "FUNCTIONAL3", "content": "구체적인 기능 요구사항"}},
      {{"requirementType": "PERFORMANCE2", "content": "구체적인 성능 요구사항 (수치 포함)"}},
      {{"requirementType": "PERFORMANCE1", "content": "구체적인 성능 요구사항 (수치 포함)"}}
    ]

    응답은 반드시 위 JSON 배열로만 시작하고 끝나야 합니다.

    
    """
    try:
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[
                {"role": "system", "content": OPTIMIZED_SYSTEM_PROMPT},
                {"role": "user", "content": enhanced_prompt}
            ],
            max_tokens=4000,
            temperature=0.3
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f"오류 발생: {e}"

In [9]:
detail_lst = []
for text, content in zip(texts, contents) :
  result = json.loads(detail_plus(text, content))
  detail_lst.append(result)
  
df['detail'] = detail_lst 

In [45]:
df = df.drop(columns = ["Unnamed: 0"])
df.to_csv("data_detail_plus.csv", index = True, index_label= 'text')

KeyError: "['Unnamed: 0'] not found in axis"

In [44]:
df = pd.read_csv("data_detail_plus.csv")
df.head()

,text,text.1,contents,relationships_data,ERD_data,API_specs_data,detail
0,0,"GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹...","{'project_summary': {'title': 'GQ 서비스', 'categ...","[{'from': 'User', 'to': 'StudyGroup', 'type': ...","[{'name': 'User', 'attributes': [{'name': 'use...","{'api_specification': {'openapi': '3.0.0', 'in...","[{'requirementType': 'FUNCTIONAL1', 'content':..."
1,1,‘기억박물관’은 치매 환자와 그 가족을 위한 따뜻한 기억 회복 웹사이트입니다. 사용...,"{'project_summary': {'title': '기억박물관', 'catego...","[{'from': 'User', 'to': 'Photo', 'type': 'one-...","[{'name': 'User', 'attributes': [{'name': 'use...","{'api_specification': {'openapi': '3.0.0', 'in...","[{'requirementType': 'FUNCTIONAL1', 'content':..."
2,2,"이 앱은 보행 약자들을 위한 지하철 길찾기 앱으로, 계단 없이 이동할 수 있는 경로...",{'project_summary': {'title': '보행 약자를 위한 지하철 길...,"[{'from': 'User', 'to': 'Route', 'type': 'one-...","[{'name': 'User', 'attributes': [{'name': 'use...","{'api_specification': {'openapi': '3.0.0', 'in...","[{'requirementType': 'FUNCTIONAL1', 'content':..."
3,3,"이 앱은 1인 가구를 위한 맞춤형 플랫폼으로, 주로 커뮤니티 기능에 중점을 두고 있...","{'project_summary': {'title': '1인 가구 맞춤형 플랫폼',...","[{'from': 'User', 'to': 'Post', 'type': 'one-t...","[{'name': 'User', 'attributes': [{'name': 'use...","{'api_specification': {'openapi': '3.0.0', 'in...","[{'requirementType': 'FUNCTIONAL1', 'content':..."
4,4,"이 웹앱은 응급실 및 병원 찾기 기능을 제공하여, 응급 상황에서 사용자가 빠르고 안...",{'project_summary': {'title': '응급실 및 병원 찾기 웹앱'...,"[{'from': 'User', 'to': 'EmergencyRoom', 'type...","[{'name': 'User', 'attributes': [{'name': 'use...","{'api_specification': {'openapi': '3.0.0', 'in...","[{'requirementType': 'FUNCTIONAL1', 'content':..."
